In [1]:
import os

import numpy as np
import pandas as pd
import polars as pl

from scipy.optimize import minimize, Bounds

from metric import score


In [2]:

MIN_INVESTMENT = 0
MAX_INVESTMENT = 2

In [3]:
train = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv", index_col="date_id")

In [4]:
solution = train.loc[8810:8990, ["forward_returns", "risk_free_rate"]]

def fun(x):
    submission = pd.DataFrame({"prediction": x}, index=solution.index)
    return - score(solution, submission, None)

In [5]:
res = minimize(
    fun,
    x0=np.full(solution.shape[0], 0.1),
    method="Powell",
    bounds=Bounds(lb=MIN_INVESTMENT, ub=MAX_INVESTMENT),
    tol=1e-9
)

In [6]:
solution["prediction"] = res.x
solution

,forward_returns,risk_free_rate,position,strategy_returns,prediction
date_id,,,,,
8810,0.005988,0.000167,9.836446e-02,0.000740,9.836446e-02
8811,0.011115,0.000167,5.230580e-02,0.000740,5.230580e-02
8812,0.000067,0.000167,4.547755e-10,0.000167,4.547755e-10
8813,-0.010526,0.000167,4.547755e-10,0.000167,4.547755e-10
8814,-0.011412,0.000166,4.547755e-10,0.000166,4.547755e-10
...,...,...,...,...,...
8986,0.002312,0.000156,2.708809e-01,0.000740,2.708809e-01
8987,0.002891,0.000156,2.133642e-01,0.000740,2.133642e-01
8988,0.008310,0.000156,7.161090e-02,0.000740,7.161090e-02


In [7]:
import kaggle_evaluation.default_inference_server

def predict(test: pl.DataFrame) -> pl.DataFrame:
    test_df = test.to_pandas().set_index("date_id")
    test_df = test_df.join(solution)
    test = test.with_columns(
        prediction = test_df["prediction"].fillna(0.0).values
    )
    return test


inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway(("/kaggle/input/hull-tactical-market-prediction/",))